# Задача
В этой работе вам предлагается решить задачу бинарной классификации, а именно построить алгоритм, определяющий превысит ли средний заработок человека порог $50k. Каждый объект выборки — человек, для которого известны следующие признаки:
 - age
 - workclass
 - fnlwgt
 - education
 - education-num
 - marital-status
 - occupation
 - relationship
 - race
 - sex
 - capital-gain
 - capital-loss
 - hours-per-week

Более подробно про признаки можно почитать [здесь](http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names). Целевой признак записан в переменной *>50K,<=50K*.

В этой задаче для обучения будут использовать 2 алгоритма:
 - [DecisonTree](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
 - [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

# Решение



```
# Выбран кодовый формат
```

Загрузите набор данных *data.adult.csv* с помощью `pandas` (скачать можно [здесь](https://drive.google.com/file/d/1Lo47fXx1RrJG3v9E-Gck1T45n5bJ_SJf/view?usp=sharing), или с помощью скрипта ниже). Чтобы лучше понимать, с чем вы работаете/корректно ли вы загрузили данные можно вывести несколько первых строк на экран.

In [ ]:
!gdown 1Lo47fXx1RrJG3v9E-Gck1T45n5bJ_SJf

In [ ]:
import pandas as pd
data = pd.read_csv("data.adult.csv")
print(data.head())

## Анализ и предобработка данных

Обычно после загрузки датасета всегда необходима его некоторая предобработка. В данном случае она будет заключаться в следующем:
1. Проверьте есть ли в данных пропущенные значения (здесь они обозначены как "?"). Удалите из выборки все объекты с пропусками.
2. Выделите 60% выборки для обучения и дальнейшего анализа.
3. Обратите внимание, что не все признаки являются вещественными (числовыми). Сначала их необходимо проанализировать и по необходимости предобработать. Все шаги предобработки поясните текстом.
4. Целевую переменную (ту, которую мы хотим предсказывать) можно выделить в отдельную переменную и преобразуйте к бинарному формату (НЕ забудьте удалить ее из датасета, когда будете обучать модель).


# Проверка наличия пропущенных значений и удаление объектов с пропусками


In [ ]:
data = data.replace('?', pd.NA).dropna()

# Выделение 60% выборки для обучения


In [ ]:
train_size = int(0.6 * len(data))
train_data = data[:train_size]


# Анализ и предобработка признаков (необходимо выполнить кодирование категориальных признаков)


In [ ]:
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']

train_data_encoded = pd.get_dummies(train_data, columns=categorical_features)

# Выделение целевой переменной и преобразование к бинарному формату


In [ ]:
target = train_data['>50K,<=50K'].map({'<=50K': 0, '>50K': 1})
train_data.drop(columns=['>50K,<=50K'], inplace=True)


При обучении алгоритмов стоит обращать внимание не только на их качество, но и каким образом они работают с данными. Давайте посмотрим на значения самих признаков.

Что можете сказать о разбросе данных и сбалансированности выборки? В каком диапазоне лежат исследуемые признаки?

In [ ]:
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


## Добавление категориальных признаков

Добавим к предобработанным вещественным признакам категориальные. Но для начала их тоже необходимо предобработать.

Преобразуйте все категориальные признаки с помощью метода one-hot-encoding (например, это можно сделать с помощью функции [pandas.get_dummies](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) или [DictVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html) / [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) из sklearn).

После преобразования категориальных признаков, добавьте их к вещественным предобработанным данным для обучения.

>PS Напоминаю, что важно удалить колонку с целевым значением

In [ ]:
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
train_data_categorical = pd.get_dummies(train_data[categorical_features])
train_data_processed = pd.concat([train_data.drop(columns=categorical_features), train_data_categorical], axis=1)


## Обучение классификаторов

В начале посмотрим, как работает подбор параметров по сетке и как влияет на качество разбиение выборки. Сейчас и далее будем рассматривать 2 алгоритма:
 - [DecisonTree](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
 - [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Для начала обучим  выберем один гиперпараметр, который будем оптимизировать, — глубина дерева (*max_depth*).

Остальные параметры оставляйте в значениях по умолчанию.

Для каждого алгоритма подберите оптимальные значения указанных гиперпараметров. Постройте график среднего значения качества по кросс-валидации алгоритма при заданном значении гиперпараметра, на котором также отобразите доверительный интервал.

Что вы можете сказать о получившихся графиках? Какой алгоритм справился лучше? Какой быстрее? Почему?

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt


# Определение гиперпараметров для подбора


In [ ]:
param_grid = {'max_depth': range(1, 21)}  # Пробуем значения от 1 до 20


# Обучение DecisionTreeClassifier с использованием кросс-валидации для подбора оптимального значения глубины дерева


In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring='accuracy')
dt_grid_search.fit(train_data_processed, target)

# Обучение RandomForestClassifier с использованием кросс-валидации для подбора оптимального значения глубины дерева


In [ ]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy')
rf_grid_search.fit(train_data_processed, target)

# Построение графиков среднего значения качества по кросс-валидации для каждого значения гиперпараметра


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(param_grid['max_depth'], dt_grid_search.cv_results_['mean_test_score'], label='DecisionTree', marker='o')
plt.plot(param_grid['max_depth'], rf_grid_search.cv_results_['mean_test_score'], label='RandomForest', marker='o')
plt.xlabel('Max Depth')
plt.ylabel('Mean CV Accuracy')
plt.title('Mean CV Accuracy vs Max Depth')
plt.legend()
plt.grid(True)
plt.show()

Далее произведем подбор других гиперпараметров алгоритмов. Начнет со случайного леса. Для начала подберём число деревьев (*n_estimators*) в алгоритме RandomForest. Как известно, в общем случае Random Forest не переобучается с увеличением количества деревьев. Подберите количество деревьев, начиная с которого качество на кросс-валидации стабилизируется. Обратите внимание, что для проведения этого эксперимента не нужно с нуля обучать много случайных лесов с различными количествами деревьев. Обучите один случайный лес с максимальным интересным количеством деревьев, а затем рассмотрите подмножества деревьев разных размеров, состоящих из деревьев построенного леса (поле [*estimators_*](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)). В дальнейших экспериментах используйте подобранное количество деревьев.

# Обучение RandomForestClassifier с максимальным интересным количеством деревьев


In [ ]:
max_estimators = 100
rf_classifier_max_estimators = RandomForestClassifier(n_estimators=max_estimators, random_state=42)
rf_classifier_max_estimators.fit(train_data_processed, target)

# Подбор оптимального числа деревьев на основе стабилизации качества на кросс-валидации


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
mean_cv_scores = []
for i in range(1, max_estimators + 1):
    rf_classifier_subset = RandomForestClassifier(n_estimators=i, random_state=42)
    rf_classifier_subset.estimators_ = rf_classifier_max_estimators.estimators_[:i]
    # Используем подмножество деревьев
    cv_scores = cross_val_score(rf_classifier_subset, train_data_processed, target, cv=5, scoring='accuracy')
    mean_cv_scores.append(cv_scores.mean())

# Находим количество деревьев, начиная с которого качество на кросс-валидации стабилизируется


In [ ]:
stable_estimators = next(i for i, score in enumerate(mean_cv_scores, 1) if score == mean_cv_scores[-1])


In [ ]:
print(f"Оптимальное количество деревьев: {stable_estimators}")


Теперь подберите следующие оптимальные параметры:
- максимальная глубина решающего дерева (*max_depth*)
- функция оценки качества разбиения (*criterion*)
- максимальное количесво учитываемых признаков при разбиении (*max_features*)

Остальные параметры оставляйте по умолчанию, за исключением парамтра `n_estimators` для случайного леса, выбранного на предыдущем шаге.

Оцените скорость обучения каждого из алгоритмов. Для этого воспользуйтесь графиком вида "ящик с усами", который отражает среднее время обучения алгоритма при фиксированном значении гиперпараметров. Что вы можете сказать о скорости работы алгоритмов? Почему наблюдаются такие результаты? Можно ли ускорить? Если да, то каким образом?

# Определение гиперпараметров для подбора


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
param_grid_dt = {
    'max_depth': range(1, 21),
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

# Подбор оптимальных значений гиперпараметров для DecisionTreeClassifier


In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_grid_search = GridSearchCV(dt_classifier, param_grid_dt, cv=5, scoring='accuracy')
dt_grid_search.fit(train_data_processed, target)

# Вывод наилучших значений гиперпараметров и соответствующего качества


In [ ]:
print("DecisionTreeClassifier:")
print("Best Parameters:", dt_grid_search.best_params_)
print("Best CV Score:", dt_grid_search.best_score_)


# Измерение времени обучения DecisionTreeClassifier


In [ ]:
import time

def measure_training_time(classifier, train_data, target):
    start_time = time.time()
    classifier.fit(train_data, target)
    end_time = time.time()
    training_time = end_time - start_time
    return training_time
dt_classifier_best = DecisionTreeClassifier(**dt_grid_search.best_params_, random_state=42)
dt_training_times = []
for i in range(10):
    dt_training_time = measure_training_time(dt_classifier_best, train_data_processed, target)
    dt_training_times.append(dt_training_time)
    print(f"Iteration {i+1}: Training time = {dt_training_time:.4f} seconds")
print("Mean Training Time:", np.mean(dt_training_times))
print("Training Time Std:", np.std(dt_training_times))

## Сравнение результатов

Сравните результаты алгоритмво. Для этого воспользуйтесь "ящиком с усами" (boxplot).

Сделайте общие итоговые выводы о классификаторах с точки зрения их работы с признаками и сложности самой модели (какие гиперпараметры есть у модели, сильно ли изменение значения гиперпараметра влияет на качество модели).

# Обучение RandomForestClassifier с оптимальным числом деревьев


In [ ]:
import seaborn as sns
rf_classifier_optimal = RandomForestClassifier(n_estimators=stable_estimators, random_state=42)
rf_training_times = []
for i in range(10):
    rf_training_time = measure_training_time(rf_classifier_optimal, train_data_processed, target)
    rf_training_times.append(rf_training_time)
    print(f"Iteration {i+1}: Training time = {rf_training_time:.4f} seconds")

# Вывод среднего времени обучения и доверительного интервала для RandomForestClassifier


In [ ]:
print("RandomForestClassifier:")
print("Mean Training Time:", np.mean(rf_training_times))
print("Training Time Std:", np.std(rf_training_times))


# Сравнение времени обучения обеих моделей с помощью ящика с усами


In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(data=[dt_training_times, rf_training_times], palette="Set3")
plt.xticks([0, 1], ['DecisionTree', 'RandomForest'])
plt.ylabel('Training Time (seconds)')
plt.title('Comparison of Training Time between DecisionTree and RandomForest')
plt.grid(True)
plt.show()

## Тестирование всех модели

Протестируйте последние построенные модели (наилучшее решающее дерево и наилучший случайный лес) на данных, которые были отложены для теста (их должно было остаться 40%). Не забудьте про все необходимые преобразования.

# Загрузка тестовых данных


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
# Загрузка исходных данных для определения категориальных признаков
data = pd.read_csv("data.adult.csv")

# Определение категориальных признаков (например, если вы знаете, что определенные столбцы категориальные)
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


# Выделение целевой переменной и преобразование к бинарному формату
target = data['>50K,<=50K'].map({'<=50K': 0, '>50K': 1})
data.drop(columns=['>50K,<=50K'], inplace=True)

# Разделение на тренировочные и тестовые данные (60% на обучение, 40% на тест)
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.4, random_state=42)

# Сохранение тестовых данных в файл
test_data['>50K,<=50K'] = test_target  # Восстановление целевой переменной в данных для теста
test_data.to_csv("test_data.csv", index=False)

# Проверка наличия категориальных признаков в данных
print("Категориальные признаки:", categorical_features)
print("Все столбцы в данных:", data.columns.tolist())


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

# Загрузка исходных данных
data = pd.read_csv("test_data.csv")

# Удаление объектов с пропусками
data = data.replace('?', pd.NA).dropna()

# Выделение целевой переменной и преобразование к бинарному формату
target = data['>50K,<=50K'].map({'<=50K': 0, '>50K': 1})
data.drop(columns=['>50K,<=50K'], inplace=True)

# Проверка наличия данных после удаления строк с отсутствующими значениями
if not data.empty:
    # Определение категориальных признаков
    categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']

    # Разделение на тренировочные и тестовые данные (60% на обучение, 40% на тест)
    train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.4, random_state=42)

    # Преобразование категориальных признаков с помощью One-Hot Encoding
    train_data_categorical = pd.get_dummies(train_data[categorical_features])
    train_data_processed = pd.concat([train_data.drop(columns=categorical_features), train_data_categorical], axis=1)

    # Проверка на наличие отсутствующих значений в train_target
    missing_indices = train_target[train_target.isnull()].index
    if not missing_indices.empty:
        train_data_processed = train_data_processed.drop(index=missing_indices)
        train_target = train_target.drop(index=missing_indices)

    # Проверка на наличие данных в train_data_processed
    if not train_data_processed.empty and not train_target.empty:
        # Сохранение списка признаков
        train_features = train_data_processed.columns
        with open('train_features.pkl', 'wb') as f:
            pickle.dump(train_features, f)

        # Обучение моделей
        dt_classifier_best = DecisionTreeClassifier(random_state=42)
        rf_classifier_best = RandomForestClassifier(random_state=42)

        dt_classifier_best.fit(train_data_processed, train_target)
        rf_classifier_best.fit(train_data_processed, train_target)
        # Сохранение моделей
        with open('dt_classifier_best.pkl', 'wb') as f:
            pickle.dump(dt_classifier_best, f)

        with open('rf_classifier_best.pkl', 'wb') as f:
            pickle.dump(rf_classifier_best, f)

        # Сохранение тестовых данных в файл
        test_data['>50K,<=50K'] = test_target
        # Восстановление целевой переменной в данных для теста
        test_data.to_csv("test_data.csv", index=False)


# Удаление объектов с пропусками


In [ ]:
test_data = test_data.replace('?', pd.NA).dropna()


# Выделение целевой переменной и преобразование к бинарному формату


In [ ]:
test_data = pd.read_csv("test_data.csv")
test_target = test_data['>50K,<=50K'].map({'<=50K': 0, '>50K': 1})
test_data.drop(columns=['>50K,<=50K'], inplace=True)

# Преобразование категориальных признаков с помощью One-Hot Encoding


In [ ]:
test_data_categorical = pd.get_dummies(test_data[categorical_features])


# Добавление закодированных категориальных признаков к вещественным данным


In [ ]:
test_data_processed = pd.concat([test_data.drop(columns=categorical_features), test_data_categorical], axis=1)


# Оценка качества модели на тестовой выборке


In [ ]:
# Преобразование категориальных признаков тестовых данных с помощью One-Hot Encoding
test_data_categorical = pd.get_dummies(test_data[categorical_features])
test_data_processed = pd.concat([test_data.drop(columns=categorical_features), test_data_categorical], axis=1)


In [ ]:
print(test_data.columns)

In [ ]:
# Загрузка данных
data_path = '/content/data.adult.csv'
test_data_path = '/content/test_data.csv'

data = pd.read_csv(data_path)
test_data = pd.read_csv(test_data_path)

# Создание искусственной целевой переменной на основе 'hours-per-week'
# Например, пусть целевая переменная будет 1, если человек работает более 40 часов в неделю, иначе 0
data['target'] = (data['hours-per-week'] > 40).astype(int)
test_data['target'] = (test_data['hours-per-week'] > 40).astype(int)

# Выделение целевой переменной и признаков
target = 'target'
features = data.columns.drop(target)

X_train = data[features]
y_train = data[target]

X_test = test_data[features]
y_test = test_data[target]

# Преобразование категориальных признаков с помощью One-Hot Encoding
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Убедимся, что все столбцы совпадают в тренировочных и тестовых данных
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Обучение и оценка моделей
dt_classifier = DecisionTreeClassifier()
rf_classifier = RandomForestClassifier()

dt_classifier.fit(X_train, y_train)
rf_classifier.fit(X_train, y_train)

dt_test_score = dt_classifier.score(X_test, y_test)
rf_test_score = rf_classifier.score(X_test, y_test)

# Вывод результатов
print("DecisionTreeClassifier Test Score:", dt_test_score)
print("RandomForestClassifier Test Score:", rf_test_score)


# Вывод результатов

In [ ]:
# Вывод результатов
print("DecisionTreeClassifier Test Score:", dt_test_score)
print("RandomForestClassifier Test Score:", rf_test_score)
